In this section we are going to deal with automated feature engineering using Featuretools. It is an opensource python library

The concept of Deep Feature Synthesis is to use basic building blocks known as feature primitives (like the transformations and aggregations done above) that can be stacked on top of each other to form new features. The depth of a "deep feature" is equal to the number of stacked primitives.

    step 1: defining entities (tables) in an EntitySet
    step 2: definiing the relationship between the entities 
    step 3: defining premitive features
    
we can see how useful featuretools is: it performed the same operations we did manually but also many more in addition. Examining the names of the features in the dataframe brings us to the final piece of the puzzle: deep features.

Explaining what these features are:
        
    MEAN(loans.loan_amount): If client A has 1000 loans then this feature will represent the mean of all the loans

In [1]:
#! pip install featuretools

In [36]:
import pandas as pd
import numpy as np

import featuretools as ft
import warnings 

pd.options.display.max_colwidth = 200


In [3]:
clients = pd.read_csv('clients.csv', parse_dates = ['joined'])
loans = pd.read_csv('loans.csv', parse_dates = ['loan_start', 'loan_end'])
payments = pd.read_csv('payments.csv', parse_dates = ['payment_date'])

In [4]:
print(clients.columns)
print(loans.columns)
print(payments.columns)

Index(['client_id', 'joined', 'income', 'credit_score'], dtype='object')
Index(['client_id', 'loan_type', 'loan_amount', 'repaid', 'loan_id',
       'loan_start', 'loan_end', 'rate'],
      dtype='object')
Index(['loan_id', 'payment_amount', 'payment_date', 'missed'], dtype='object')


In [5]:
stats_table = loans.groupby('client_id')['loan_amount'].agg(['mean', 'max', 'min'])
stats_table.columns = ['mean_loan_amount', 'max_loan_amount', 'min_loan_amount']

In [6]:
stats_table.head()

,mean_loan_amount,max_loan_amount,min_loan_amount
client_id,,,
2003,5010.666667,10314,2267
2004,11882.000000,12651,10464
2011,12006.000000,13339,9837
2031,7311.300000,13975,772
2041,7054.142857,14024,1049


In [7]:
print(len(clients['client_id'].unique()))
print(len(stats_table))
print(len(loans['client_id'].unique()))

241
232
232


In [8]:
clients_manual = clients.merge(stats_table, left_on = 'client_id', right_index=True, how = 'left')
clients_manual.head(10)

,client_id,joined,income,credit_score,mean_loan_amount,max_loan_amount,min_loan_amount
0,2697,2013-02-14,45903,507,7016.600000,14927.0,1262.0
1,4647,2005-06-23,47037,628,7374.700000,13708.0,807.0
2,3230,2002-05-06,35246,555,7531.352941,14406.0,705.0
3,3214,2006-08-26,42311,669,6077.105263,13298.0,852.0
4,2265,2000-08-28,40387,679,7562.055556,14804.0,587.0
5,2483,2013-05-19,49188,511,5911.052632,13875.0,934.0
6,3092,2005-05-15,48432,766,7485.947368,14769.0,726.0
7,3441,2001-06-10,45021,746,8079.150000,13985.0,630.0
8,2489,2002-04-17,42915,655,7740.000000,14306.0,1231.0
9,2909,2011-07-03,49407,503,7828.666667,14574.0,1428.0


In [9]:
null_columns = clients_manual.columns[clients_manual.isnull().any()]
clients_manual[null_columns].isnull().sum()

mean_loan_amount    9
max_loan_amount     9
min_loan_amount     9
dtype: int64

creating an entity which is just a table. 

        1. index: Unique identifier
        2. time_index: 
        3. make_index: True/False
If the data also has a uniquely identifying time index, we can pass that in as the time_index parameter.

In [10]:
es = ft.EntitySet(id = 'clients')
es = es.entity_from_dataframe(entity_id = 'clients', dataframe = clients, 
                              index = 'client_id', time_index = 'joined')

In [11]:
payments.columns

Index(['loan_id', 'payment_amount', 'payment_date', 'missed'], dtype='object')

In [12]:
es = es.entity_from_dataframe(entity_id = 'loans', 
                              dataframe = loans, 
                              index = 'loan_id', 
                              time_index = 'loan_start')

In [13]:
es = es.entity_from_dataframe(entity_id = 'payments', 
                              dataframe = payments,
                              make_index = True,
                              index = 'payment_id',
                              time_index = 'payment_date')

In [14]:
es

Entityset: clients
  Entities:
    clients [Rows: 241, Columns: 4]
    loans [Rows: 1811, Columns: 8]
    payments [Rows: 33708, Columns: 5]
  Relationships:
    No relationships

In [15]:
es['loans']

Entity: loans
  Variables:
    loan_id (dtype: index)
    client_id (dtype: numeric)
    loan_type (dtype: categorical)
    loan_amount (dtype: numeric)
    repaid (dtype: numeric)
    loan_start (dtype: datetime_time_index)
    loan_end (dtype: datetime)
    rate (dtype: numeric)
  Shape:
    (Rows: 1811, Columns: 8)

In [16]:
es['clients']

Entity: clients
  Variables:
    client_id (dtype: index)
    joined (dtype: datetime_time_index)
    income (dtype: numeric)
    credit_score (dtype: numeric)
  Shape:
    (Rows: 241, Columns: 4)

In [17]:
# in thsi step we are creating the relationship between client and loans
client_previous = ft.Relationship(es['clients']['client_id'],
                                    es['loans']['client_id'])

print(client_previous)

<Relationship: loans.client_id -> clients.client_id>


In [18]:
# Add the relationship to the entity set
es = es.add_relationship(client_previous)

In [19]:
es

Entityset: clients
  Entities:
    clients [Rows: 241, Columns: 4]
    loans [Rows: 1811, Columns: 8]
    payments [Rows: 33708, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id

In [20]:
payments_relation = ft.Relationship(es['loans']['loan_id'],
                             es['payments']['loan_id'])
es = es.add_relationship(payments_relation)
es


Entityset: clients
  Entities:
    clients [Rows: 241, Columns: 4]
    loans [Rows: 1811, Columns: 8]
    payments [Rows: 33708, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id
    payments.loan_id -> loans.loan_id

 ## Premitive Features
There are total of 78 premitive features, but two categories: aggregate and transform
 
 
Aggregation:  function that groups together child datapoints for each parent and then calculates a statistic such as mean, min, max, or standard deviation
 
 Transformation:  an operation applied to one or more columns in a single table. An example would be extracting the day from dates, or finding the difference between two columns in one table.

In [21]:
prim = ft.list_primitives()

In [22]:
prim

,name,type,dask_compatible,description
0,num_unique,aggregation,True,"Determines the number of distinct values, ignoring `NaN` values."
1,last,aggregation,False,Determines the last value in a list.
2,n_most_common,aggregation,False,Determines the `n` most common elements.
3,entropy,aggregation,False,Calculates the entropy for a categorical variable
4,count,aggregation,True,"Determines the total number of values, excluding `NaN`."
...,...,...,...,...
74,not,transform,True,Negates a boolean value.
75,is_weekend,transform,True,Determines if a date falls on a weekend.
76,divide_numeric_scalar,transform,True,Divide each element in the list by a scalar.
77,equal_scalar,transform,True,Determines if values in a list are equal to a given scalar.


In [23]:
prim[prim['type'] == 'aggregation'].head(10)

,name,type,dask_compatible,description
0,num_unique,aggregation,True,"Determines the number of distinct values, ignoring `NaN` values."
1,last,aggregation,False,Determines the last value in a list.
2,n_most_common,aggregation,False,Determines the `n` most common elements.
3,entropy,aggregation,False,Calculates the entropy for a categorical variable
4,count,aggregation,True,"Determines the total number of values, excluding `NaN`."
5,mode,aggregation,False,Determines the most commonly repeated value.
6,first,aggregation,False,Determines the first value in a list.
7,min,aggregation,True,"Calculates the smallest value, ignoring `NaN` values."
8,all,aggregation,True,Calculates if all values are 'True' in a list.
9,sum,aggregation,True,"Calculates the total addition, ignoring `NaN`."


In [37]:
prim[prim['type'] == 'transform'].head(10)

,name,type,dask_compatible,description
22,diff,transform,False,Compute the difference between the value in a list and the
23,cum_max,transform,False,Calculates the cumulative maximum.
24,less_than_scalar,transform,True,Determines if values are less than a given scalar.
25,or,transform,True,Element-wise logical OR of two lists.
26,isin,transform,True,Determines whether a value is present in a provided list.
27,day,transform,True,Determines the day of the month from a datetime.
28,multiply_boolean,transform,True,Element-wise multiplication of two lists of boolean values.
29,modulo_numeric_scalar,transform,True,Return the modulo of each element in the list by a scalar.
30,month,transform,True,Determines the month value of a datetime.
31,weekday,transform,True,Determines the day of the week from a datetime.


In [32]:
prim_transformative_name = prim[prim['type'] == 'transform']['name'].values
print(prim_transformative_name)

['diff' 'cum_max' 'less_than_scalar' 'or' 'isin' 'day' 'multiply_boolean'
 'modulo_numeric_scalar' 'month' 'weekday' 'cum_count' 'less_than'
 'second' 'absolute' 'and' 'multiply_numeric_scalar' 'latitude' 'week'
 'modulo_numeric' 'scalar_subtract_numeric_feature'
 'less_than_equal_to_scalar' 'percentile' 'cum_min' 'divide_by_feature'
 'subtract_numeric' 'less_than_equal_to' 'negate' 'hour' 'greater_than'
 'year' 'divide_numeric' 'greater_than_equal_to' 'add_numeric'
 'num_characters' 'time_since_previous' 'not_equal_scalar' 'num_words'
 'minute' 'cum_sum' 'greater_than_equal_to_scalar' 'add_numeric_scalar'
 'longitude' 'cum_mean' 'modulo_by_feature' 'multiply_numeric' 'equal'
 'haversine' 'is_null' 'time_since' 'not_equal' 'subtract_numeric_scalar'
 'greater_than_scalar' 'not' 'is_weekend' 'divide_numeric_scalar'
 'equal_scalar' 'age']


### Making our own features
Using primitives is surprisingly easy using the ft.dfs function (which stands for deep feature synthesis). In this function, we specify the entityset to use; the target_entity, which is the dataframe we want to make the features for (where the features end up); the agg_primitives which are the aggregation feature primitives; and the trans_primitives which are the transformation primitives to apply.

In [33]:
# Create new features using specified primitives
features, feature_names = ft.dfs(entityset = es, target_entity = 'clients', 
                                 agg_primitives = ['mean', 'max', 'percent_true', 'last'],
                                 trans_primitives = ['month', 'diff','divide_numeric','year'])

C:\Users\Sidhant\Anaconda3\lib\site-packages\featuretools\synthesis\dfs.py:313: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['percent_true']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


In [26]:
pd.DataFrame(features['MONTH(joined)'].head())

,MONTH(joined)
client_id,
2447,1
2963,2
3294,3
3895,3
3469,4


In [34]:
features.head()

,income,credit_score,MEAN(loans.loan_amount),MEAN(loans.rate),MEAN(loans.repaid),MAX(loans.loan_amount),MAX(loans.rate),MAX(loans.repaid),LAST(loans.loan_amount),LAST(loans.loan_id),...,income / MAX(payments.missed),income / MAX(payments.payment_amount),income / MEAN(loans.loan_amount),income / MEAN(loans.rate),income / MEAN(loans.repaid),income / MEAN(payments.missed),income / MEAN(payments.payment_amount),YEAR(LAST(loans.loan_end)),YEAR(LAST(loans.loan_start)),YEAR(LAST(payments.payment_date))
client_id,,,,,,,,,,,,,,,,,,,,,
2447,41501,546,11291.666667,5.483333,0.666667,14986.0,10.16,1.0,10116.0,10285.0,...,41501.0,15.036594,3.675365,7568.571429,62251.50,69168.333333,32.409635,2016.0,2014.0,2015.0
2963,47705,735,8586.000000,2.782000,0.800000,14759.0,7.16,1.0,13700.0,11110.0,...,47705.0,19.013551,5.556138,17147.735442,59631.25,98301.212121,46.065606,2016.0,2014.0,2015.0
3294,38544,769,12812.500000,3.500000,0.500000,13768.0,3.56,1.0,13768.0,10890.0,...,38544.0,14.275556,3.008312,11012.571429,77088.00,100214.400000,19.222465,2012.0,2010.0,2010.0
3895,39890,677,5542.285714,3.731429,0.571429,11475.0,5.17,1.0,11475.0,11900.0,...,39890.0,16.669453,7.197391,10690.275651,69807.50,68900.909091,40.911716,2009.0,2008.0,2011.0
3469,46552,607,8128.777778,4.211111,0.222222,14373.0,7.65,1.0,2996.0,10397.0,...,46552.0,16.328306,5.726814,11054.564644,209484.00,83452.975610,39.472238,2015.0,2013.0,2014.0


In [35]:
feature_names

[<Feature: income>,
 <Feature: credit_score>,
 <Feature: MEAN(loans.loan_amount)>,
 <Feature: MEAN(loans.rate)>,
 <Feature: MEAN(loans.repaid)>,
 <Feature: MAX(loans.loan_amount)>,
 <Feature: MAX(loans.rate)>,
 <Feature: MAX(loans.repaid)>,
 <Feature: LAST(loans.loan_amount)>,
 <Feature: LAST(loans.loan_id)>,
 <Feature: LAST(loans.loan_type)>,
 <Feature: LAST(loans.rate)>,
 <Feature: LAST(loans.repaid)>,
 <Feature: MEAN(payments.missed)>,
 <Feature: MEAN(payments.payment_amount)>,
 <Feature: MAX(payments.missed)>,
 <Feature: MAX(payments.payment_amount)>,
 <Feature: LAST(payments.missed)>,
 <Feature: LAST(payments.payment_amount)>,
 <Feature: LAST(payments.payment_id)>,
 <Feature: MONTH(joined)>,
 <Feature: DIFF(credit_score)>,
 <Feature: DIFF(income)>,
 <Feature: credit_score / income>,
 <Feature: income / credit_score>,
 <Feature: YEAR(joined)>,
 <Feature: MEAN(loans.DIFF(loan_amount))>,
 <Feature: MEAN(loans.DIFF(rate))>,
 <Feature: MEAN(loans.DIFF(repaid))>,
 <Feature: MEAN(loans.L

In [38]:
pd.DataFrame(features['MEAN(loans.loan_amount)'].head(10))

,MEAN(loans.loan_amount)
client_id,
2447,11291.666667
2963,8586.000000
3294,12812.500000
3895,5542.285714
3469,8128.777778
2265,7562.055556
3432,NaN
4829,8279.722222
3567,7507.250000


In [39]:
loans.loc[loans['client_id'] == 2447]

,client_id,loan_type,loan_amount,repaid,loan_id,loan_start,loan_end,rate
1365,2447,cash,10116,0,10285,2014-12-03,2016-07-13,2.63
1366,2447,home,14986,1,11983,2005-04-05,2007-09-01,10.16
1367,2447,cash,8773,1,10222,2001-01-08,2003-01-30,3.66


In [44]:
payments.loc[payments['loan_id'] == 10285]

,payment_id,loan_id,payment_amount,payment_date,missed
16131,16131,10285,1206,2015-01-14,1
16132,16132,10285,1793,2015-02-04,0
16133,16133,10285,1675,2015-03-10,1
16134,16134,10285,1716,2015-04-26,1
16135,16135,10285,1206,2015-05-28,1
31559,31559,10285,778,2014-06-10,1
31560,31560,10285,1265,2014-06-29,1
31561,31561,10285,899,2014-08-16,1
31562,31562,10285,1149,2014-09-22,0
31563,31563,10285,798,2014-10-20,0


In [40]:
pd.DataFrame(features['LAST(loans.MEAN(payments.payment_amount))'].head(10))


,LAST(loans.MEAN(payments.payment_amount))
client_id,
2447,1244.818182
2963,1165.909091
3294,2377.000000
3895,1711.666667
3469,445.142857
2265,969.821429
3432,NaN
4829,1016.550000
3567,1157.407407
